# n分割/大元からランダムにn件選びそこから更にわざと誤採点を作る（位置指定なし）


# クラスタリングを行い,学習データを選ぶ


In [2]:
import pandas as pd
import numpy as np
import copy

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

# 引数は(解答の文章ベクトル、解答の文章、クラスタ数)
# 学習データになるインデックスが先に格納され、そのあとに開発orテストデータとなるインデックスが格納されたリストを返す
def clustering(cls_token_vec, answer_text, cluster_num):
    # データ数を確認する
    student_num = len(answer_text)
    print("問題数　" + str(student_num))
    
    if cluster_num < 1:
        
        clustering_method = 1
        
        # 解答文ベクトルをpandasのデータフレームに格納
        df = pd.DataFrame(cls_token_vec)

        # 階層型クラスタリングの実施
        if clustering_method == 0:
            # ウォード法 x ユークリッド距離
            print("---Ward法でクラスタリング---\n")
            linkage_result = linkage(df, method='ward', metric='euclidean')
        elif clustering_method == 1:
            print("---最短距離法でクラスタリング---\n")
            linkage_result = linkage(df, method='single', metric='euclidean')
        elif clustering_method == 2: 
            print("---最長距離法でクラスタリング---\n")
            linkage_result = linkage(df, method='complete', metric='euclidean')
        elif clustering_method == 3: 
            print("---郡平均法でクラスタリング---\n")
            linkage_result = linkage(df, method='average', metric='euclidean')

        # クラスタ分けする閾値を決める
        print("クラスタリングの閾値d(ユークリッド距離)の倍率を入力してください(0<d≦1.0)")
        #thresh = float(input('>>>  '))
        #thresh = 0.05
        thresh = cluster_num
        print("thresh = " + str(thresh))
        threshold = thresh * np.max(linkage_result[:, 2])

        # 階層型クラスタリングの可視化
        plt.figure(num=None, figsize=(6, 3), dpi=300, facecolor='w', edgecolor='k')
        dendrogram(linkage_result, labels=answer_text, color_threshold=threshold)
        plt.show()
        #plt.savefig('figure_dendrogram.png')

        # クラスタリング結果のクラスタ番号を取得
        clustered = fcluster(linkage_result, threshold, criterion='distance')

        # クラスタリング結果を表示
        cluster_num = np.max(clustered)
        
    else:
        print("---k-means法でクラスタリング---\n")
        # クラスタ番号は0から
        
        
        clustered = KMeans(n_clusters=cluster_num, init='k-means++', n_init=10).fit_predict(cls_token_vec)   # k-means++で初期位置を決める

    # クラスタ番号が同じインデックスを集める
    cluster_index = [[] for _ in range(cluster_num)]
    for index, num in enumerate(clustered):
        cluster_index[num].append(index)

    return cal_center(cls_token_vec ,cluster_index)


In [3]:
import pdb
from sklearn.metrics import pairwise_distances
from scipy import stats
def simple(test_cls,cluster_num):
    cls_sub = np.array(test_cls)
    print("クラスタリング開始")
    clustered = KMeans(n_clusters=cluster_num, init='k-means++', n_init=10).fit_predict(cls_sub)   # k-means++で初期位置を決める
    print("クラスタリング終了")
    # クラスタ番号が同じインデックスを集める
    cluster_index = [[] for _ in range(cluster_num)]
    for index, num in enumerate(clustered):
        cluster_index[num].append(index)

    return cal_center(cls_sub.tolist() ,cluster_index)
def uncertainty(test_un,cluster_num):
    test_un = np.max(test_un,axis = 1)
    un_dict = {}
    for index,i in enumerate(test_un):
        un_dict[index] = i
    
    result = sorted(un_dict.items(), key=lambda x:x[1],reverse= False)
    print(result[:10])
    train_list = []
    test_list = []
    for i in range(cluster_num):
        train_list.append(int(result[i][0]))
    
    for i in range(len(test_un)):
        if(i not in train_list):
            test_list.append(i)
    
    random.shuffle(train_list)
    random.shuffle(test_list)
    train_list.extend(test_list)
    
    return train_list

def mergin(test_un,cluster_num):
    un_dict = {}
    test_un = np.array(test_un)
    a_argsorted = np.argsort(test_un, axis=1)
    for index,i in enumerate(test_un):
        un_dict[index] = (test_un[index][a_argsorted[index][-1]] -  test_un[index][a_argsorted[index][-2]])
    
    result = sorted(un_dict.items(), key=lambda x:x[1],reverse= False)
    print(result[:10])
    train_list = []
    test_list = []
    for i in range(cluster_num):
        train_list.append(int(result[i][0]))
    
    for i in range(len(test_un)):
        if(i not in train_list):
            test_list.append(i)
    
    random.shuffle(train_list)
    random.shuffle(test_list)
    train_list.extend(test_list)
    
    return train_list

def badge(test_un,test_cls,predict_list,cluster_num,label_num = 4):
    train_list = []
    test_list = []
    
    test_un = np.array(test_un,dtype = np.float32)
    test_cls = np.array(test_cls,dtype = np.float32)
    grad_list = []
    for index,i in enumerate(test_cls):
        #if(index % 1000 == 0):
            #print(index)
            #print("{}:{}".format(len(test_un),grad_list.shape))
        #print(grad_list.shape)
        grad_sub = []
        for j in range(label_num):
            if(j == predict_list[index]):
                grad_sub.extend(((test_un[index][j] - 1) * test_cls[index]))
            else:
                grad_sub.extend(((test_un[index][j]) * test_cls[index]))
        grad_list.append(np.array(grad_sub,dtype = np.float32))
    
    #grad_list = np.array(grad_list)
    ind = np.argmax([np.linalg.norm(s, 2) for s in grad_list])
    mu = [grad_list[ind]]
    indsAll = [ind]
    centInds = [0.] * len(grad_list)
    cent = 0
    print('#Samps\tTotal Distance')
    while len(mu) < cluster_num:
        if len(mu) == 1:
            D2 = pairwise_distances(grad_list, mu).ravel().astype(float)
        else:
            newD = pairwise_distances(grad_list, [mu[-1]]).ravel().astype(float)
            for i in range(len(grad_list)):
                if D2[i] >  newD[i]:
                    centInds[i] = cent
                    D2[i] = newD[i]
        if sum(D2) == 0.0: pdb.set_trace()
        D2 = D2.ravel().astype(float)
        Ddist = (D2 ** 2)/ sum(D2 ** 2)
        customDist = stats.rv_discrete(name='custm', values=(np.arange(len(D2)), Ddist))
        ind = customDist.rvs(size=1)[0]
        while ind in indsAll: ind = customDist.rvs(size=1)[0]
        mu.append(grad_list[ind])
        indsAll.append(ind)
        cent += 1
        
    train_list.extend(indsAll)
    for i in range(len(test_cls)):
        if(i not in train_list):
            test_list.append(i)
            
    random.shuffle(train_list)
    random.shuffle(test_list)
    train_list.extend(test_list)
    
    return train_list

def entropy(test_un,cluster_num):
    train_list = []
    test_list = []
    prob_log = np.log(test_un)
    entropy_sum = (np.array(test_un) * np.array(prob_log)).sum(1)
    prob_dict = {}
    for i in range(len(test_un)):
        prob_dict[i] = entropy_sum[i]
        
    result = sorted(prob_dict.items(), key=lambda x:x[1],reverse= False)
    
    print(result[:10])

    for i in range(cluster_num):
        train_list.append(int(result[i][0]))
    
    for i in range(len(test_un)):
        if(i not in train_list):
            test_list.append(i)
    
    random.shuffle(train_list)
    random.shuffle(test_list)
    train_list.extend(test_list)
    
    return train_list

def core_set(train_cls,test_cls,cluster_num):
    train_list = []
    test_list = []
    
    dis_list = []
    
    for i in range(len(test_cls)):
        min_dis = 1000000
        for j in range(len(train_cls)):
            dis = np.linalg.norm(test_cls[i] - np.array(train_cls[j]), ord=2)
            if(min_dis > dis):
                min_dis = dis
        dis_list.append(min_dis)
        
    train_list.append(np.argmax(dis_list))
    
    while(len(train_list) != cluster_num):
        if(len(train_list) % 100 == 0):
            print(len(train_list))
        for i in range(len(test_cls)):
            dis = np.linalg.norm(test_cls[i] - np.array(test_cls[train_list[-1]]), ord=2)
            if(dis_list[i] > dis):
                dis_list[i] = dis
                
        train_list.append(np.argmax(dis_list))
        
    for i in range(len(test_cls)):
        if(i not in train_list):
            test_list.append(i)
    
    random.shuffle(train_list)
    random.shuffle(test_list)
    train_list.extend(test_list)
    
    return train_list
def core_set_new(train_cls,test_cls,cluster_num):
    
    lb_flag = []
    for i in range(len(train_cls)):
        lb_flag.append(True)
    for i in range(len(test_cls)):
        lb_flag.append(False)
    embedding  = train_cls
    embedding.extend(test_cls)
    embedding = np.array(embedding)

    print('calculate distance matrix')
    dist_mat = np.matmul(embedding, embedding.transpose())
    sq = np.array(dist_mat.diagonal()).reshape(len(embedding), 1)
    dist_mat *= -2
    dist_mat += sq
    dist_mat += sq.transpose()
    dist_mat = np.sqrt(dist_mat)

    print('calculate greedy solution')
    mat = dist_mat[~lb_flag, :][:, lb_flag]

    for i in range(cluster_num):
        if i%100 == 0:
            print('greedy solution {}/{}'.format(i, cluster_num))
        mat_min = mat.min(axis=1)
        q_idx_ = mat_min.argmax()
        q_idx = np.arange(len(embedding))[~lb_flag][q_idx_]
        lb_flag[q_idx] = True
        mat = np.delete(mat, q_idx_, 0)
        mat = np.append(mat, dist_mat[~lb_flag, q_idx][:, None], axis=1)
        
    print(len(mat))
    print(mat)
    
def core_set_pair(train_cls,test_cls,cluster_num,device):
    
    torch.cuda.empty_cache()
    train_list = []
    test_list = []
    
    dis_list = []
    train_cls = torch.tensor(train_cls ,dtype = torch.float32,device = device)
    test_cls = torch.tensor(test_cls ,dtype = torch.float32,device = device)
    print("距離計算")
    dist_ctr = torch.cdist(test_cls,train_cls).cpu()
    #dist_ctr = pairwise_distances(np.array(test_cls), np.array(train_cls))
    
    
    
    min_dist = torch.amin(dist_ctr,1)
    
    del train_cls
    del dist_ctr
    torch.cuda.empty_cache()
    #np.amin(dist_ctr, axis=1)
    print("サンプリング開始")
    
    torch_batch = len(test_cls)
    while(len(train_list) != cluster_num):
        train_list.append(torch.argmax(min_dist).tolist())
        
        for i in range(0,len(test_cls),torch_batch):
            dist = torch.cdist(test_cls[i:i+torch_batch],test_cls[train_list[-1]:train_list[-1] + 1])
            for index,j in enumerate(torch.flatten(dist).tolist()):
                if(min_dist[i + index] > j):
                    min_dist[i + index] = j
     
    #while(len(train_list) != cluster_num):
       # print(len(train_list))
       # train_list.append(np.argmax(min_dist))
        #dist_ctr = pairwise_distances(np.array(test_cls), np.array(test_cls)[[train_list[-1]],:])
       # for i in range(len(test_cls)):
           # if(min_dist[i] > dist_ctr[i][0]):
               # min_dist[i] = dist_ctr[i][0]
    
        
    for i in range(len(test_cls)):
        if(i not in train_list):
            test_list.append(i)
    
    random.shuffle(train_list)
    random.shuffle(test_list)
    train_list.extend(test_list)
    
    del test_cls
    del dist 
    del min_dist
    
    torch.cuda.empty_cache()
    
    return train_list

# クラスタリングの重心を決める

In [4]:
import random

# 重心を求める関数 (文章ベクトル、クラスタの番号とインデックスを結びつけたリスト)
# 学習データになるインデックスが先に格納され、そのあとに開発orテストデータとなるインデックスが格納されたリストを返す
    
    
def cal_center(cls_token_vec ,cluster_index):
    
    # クラスターの中心座標（重心）を求める
    center_vec = []
    for i in range(len(cluster_index)):
        total = np.zeros_like(cls_token_vec[0])   # 解答ベクトルと同じ形の配列を用意する
        for j in cluster_index[i]:
            total += cls_token_vec[j]
        center_vec.append(total / len(cluster_index[i]))

    # 重心に一番近い解答を求める
    train_list = []
    test_list = []
    for i, vec in enumerate(center_vec):
        min_dis = 10000
        index = 0
        for j in cluster_index[i]:
            test_list.append(j)
            # 重心とのユークリッド距離を求める
            dis = np.linalg.norm(vec - cls_token_vec[j], ord=2)
            if (min_dis > dis):
                index = j
                min_dis = dis
        train_list.append(index)
        test_list.remove(index)
        
    random.shuffle(test_list)
    train_list.extend(test_list)
    
    return train_list

# 文章ベクトルの作成

In [5]:
def create_cls_token_vec(model_name,answer_text,hidden_num = 4):
    print("文章ベクトルの作成を行います")
    cls_token_vec = []
    config = AutoConfig.from_pretrained(model_name)
    config.update({"output_hidden_states": True})
    bert = AutoModel.from_pretrained(model_name,config=config).to(device)
    tokenizer_clustering = AutoTokenizer.from_pretrained(model_name)
    batch_size = 4
    for i in range(len(answer_text)//batch_size): 
        if(i<len(answer_text)//batch_size-1):
            encoded_data_clustering = tokenizer_clustering.batch_encode_plus(answer_text[batch_size*i:batch_size*(i+1)], pad_to_max_length=True, add_special_tokens=True)
            outputs = bert(input_ids = torch.tensor(encoded_data_clustering["input_ids"],device = device),attention_mask=torch.tensor(encoded_data_clustering["attention_mask"],device = device), token_type_ids=torch.tensor(encoded_data_clustering["token_type_ids"],device = device))
                           #,torch.tensor(encoded_data_clustering["token_type_ids"],device = device))
            if(hidden_num > 1):
                
                sequence_output = torch.cat([outputs["hidden_states"][-1*i][:,0] for i in range(1, hidden_num+1)], dim=1)
                cls_token_vec.extend(sequence_output.tolist())
            else:
                                                           
                outputs = outputs["hidden_states"][-2][:,0]
                cls_token_vec.extend(outputs.tolist())
        else:
            encoded_data_clustering = tokenizer_clustering.batch_encode_plus(answer_text[batch_size*i:], pad_to_max_length=True, add_special_tokens=True)
            outputs = bert(input_ids =torch.tensor(encoded_data_clustering["input_ids"],device = device),attention_mask=torch.tensor(encoded_data_clustering["attention_mask"],device = device), token_type_ids=torch.tensor(encoded_data_clustering["token_type_ids"],device = device))
            if(hidden_num > 1):
                sequence_output = torch.cat([outputs["hidden_states"][-1*i][:,0] for i in range(1, hidden_num+1)], dim=1)
                cls_token_vec.extend(sequence_output.tolist())
            else:
                outputs = outputs["hidden_states"][-2][:,0]
                cls_token_vec.extend(outputs.tolist())
    print(len(cls_token_vec))
    cls_token_vec = np.array(cls_token_vec).reshape(len(answer_text),-1).tolist()
    print("文章ベクトルの作成を終了しました")
    return cls_token_vec

def create_words_vec(model_name,answer_text,hidden_num = 1):
    print("文章ベクトルの作成を行います")
    words_vec = []
    config = AutoConfig.from_pretrained(model_name)
    config.update({"output_hidden_states": True})
    bert = AutoModel.from_pretrained(model_name,config=config).to(device)
    tokenizer_clustering = AutoTokenizer.from_pretrained(model_name)
    batch_size = 4
    for i in range(len(answer_text)//batch_size): 
        if(i<len(answer_text)//batch_size-1):
            encoded_data_clustering = tokenizer_clustering.batch_encode_plus(answer_text[batch_size*i:batch_size*(i+1)], pad_to_max_length=True, add_special_tokens=True)
            outputs = bert(torch.tensor(encoded_data_clustering["input_ids"],device = device),torch.tensor(encoded_data_clustering["attention_mask"],device = device),torch.tensor(encoded_data_clustering["token_type_ids"],device = device))
            if(hidden_num > 1):
                sequence_output = torch.cat([outputs["hidden_states"][-1*i][:,1:-1] for i in range(1, hidden_num+1)], dim=1)
                words_vec.extend(sequence_output.tolist())
            else:
                outputs = outputs["hidden_states"][-2][:,1:-1]
                words_vec.extend(outputs.tolist())
        else:
            encoded_data_clustering = tokenizer_clustering.batch_encode_plus(answer_text[batch_size*i:], pad_to_max_length=True, add_special_tokens=True)
            outputs = bert(torch.tensor(encoded_data_clustering["input_ids"],device = device),torch.tensor(encoded_data_clustering["attention_mask"],device = device),torch.tensor(encoded_data_clustering["token_type_ids"],device = device))
            if(hidden_num > 1):
                sequence_output = torch.cat([outputs["hidden_states"][-1*i][:,1:-1] for i in range(1, hidden_num+1)], dim=1)
                words_vec.extend(sequence_output.tolist())
            else:
                outputs = outputs["hidden_states"][-2][:,1:-1]
                words_vec.extend(outputs.tolist())
    print(len(words_vec))
    words_vec = np.array(words_vec).reshape(len(answer_text),-1,768).tolist()
    print("文章ベクトルの作成を終了しました")
    return words_vec

def create_cls_token_and_words_vec(model_name,answer_text,hidden_num = 1):
    print("ベクトルの作成を行います")
    cls_token_vec = []
    words_vec = []
    config = AutoConfig.from_pretrained(model_name)
    config.update({"output_hidden_states": True})
    bert = AutoModel.from_pretrained(model_name,config=config).to(device)
    tokenizer_clustering = AutoTokenizer.from_pretrained(model_name)
    for i in range(len(answer_text)): 
        encoded_data_clustering = tokenizer_clustering(answer_text[i], return_tensors="pt",add_special_tokens=True)
        outputs = bert(encoded_data_clustering["input_ids"].to(device),encoded_data_clustering["attention_mask"].to(device),encoded_data_clustering["token_type_ids"].to(device))
        if(hidden_num > 1):
            sequence_output = torch.cat([outputs["hidden_states"][-1*j][:,0] for j in range(1, hidden_num+1)], dim=1)
            word_output =  torch.cat([outputs["hidden_states"][-1*j][:,1:-1] for j in range(1, hidden_num+1)], dim=1)
        else:
            sequence_output = outputs["hidden_states"][-2][:,0]
            word_output = outputs["hidden_states"][-2][:,1:-1].view(-1,768)
        cls_token_vec.extend(sequence_output.tolist())
        words_vec.append(word_output.tolist())

    print(len(cls_token_vec))
    print(len(words_vec))
    cls_token_vec = np.array(cls_token_vec).reshape(len(answer_text),-1).tolist()
    print("ベクトルの作成を終了しました")
    return cls_token_vec,words_vec

# T-SNEを用いて次元圧縮

In [6]:
from sklearn.manifold import TSNE
def change_dimension(cls_token_vec):
    print("次元圧縮開始")
    cls_token_vec = TSNE(n_components=2, random_state=0).fit_transform(cls_token_vec)
    print("次元圧縮終了")
    return cls_token_vec

# 採点結果のアルファベットを変換

In [7]:
def str_to_num(graded_score):
    nums = []
    for score in graded_score:
        if score == "a":
            nums.append(3)
        elif score == "b":
            nums.append(2)
        elif score == "c":
            nums.append(1)
        elif score == "d":
            nums.append(0)
            
    return nums
        
def num_to_str(nums):
    graded_score = []
    for num in nums:
        #m = max(num)
        if num == 0:
            graded_score.append("d")
        elif num == 1:
            graded_score.append("c")
        elif num == 2:
            graded_score.append("b")
        elif num == 3:
            graded_score.append("a")
    
    return graded_score

def num_to_num(nums):
    result = []
    #kokokaeru
    for num in nums:
        if num == "0":
            result.append(0)
        elif num == "1":
            result.append(0)
        elif num == "2":
            result.append(0)
        elif num == "3":
            result.append(1)
        elif num == "4":
            result.append(1)
        else :
            result.append(1)
    return result
    
def num_to_num_after(nums):
    result = []
    for num in nums:
        if num == 0:
            result.append(0)
        elif num == 1:
            result.append(1)
        elif num == 2:
            result.append(2)
        elif num == 3:
            result.append(3)
        elif num == 4:
            result.append(4)
        elif num == 5:
            result.append(5)
        elif num == 6:
            result.append(6)

    return result
            
            

# 学習モデルの定義

In [8]:
from transformers import AutoConfig, AutoModel
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.utils.rnn as rnn
from transformers import AutoTokenizer,T5ForConditionalGeneration,T5EncoderModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

print(device)
class CommonLitModel(nn.Module):
    def __init__(self,MODEL_NAME,r = 4):
        super(CommonLitModel, self).__init__()
        #self.config = AutoConfig.from_pretrained(MODEL_NAME)  config=self.config self.config.hidden_size
        #self.config.update({"output_hidden_states": True})
        self.bert = AutoModel.from_pretrained(
            MODEL_NAME,
            output_hidden_states = True
        )
        self.regressor = nn.Linear(768*4, r)
        self.softmax = nn.Softmax(dim = 1)
        nn.init.normal_(self.regressor.weight,mean= 0.0,std = 0.02)
        nn.init.zeros_(self.regressor.bias)
        
    def _get_cls_vec(self, vec):
        return vec[:,0,:].view(-1, 768)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        hidden_states = outputs['hidden_states']
        vec1 = self._get_cls_vec(hidden_states[-1])
        vec2 = self._get_cls_vec(hidden_states[-2])
        vec3 = self._get_cls_vec(hidden_states[-3])
        vec4 = self._get_cls_vec(hidden_states[-4])
        
        sequence_output =  torch.cat([vec1, vec2, vec3, vec4], dim=1)  # concatenate
        logits = self.regressor(sequence_output)
        predict = self.softmax(logits)
        return logits,predict,sequence_output

/Users/xolovestephi/opt/anaconda3/envs/python38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu


# 解答文と採点結果の読み込み

In [ ]:
import time
from sklearn.metrics import cohen_kappa_score, mean_squared_error,accuracy_score
from transformers import AdamW
from transformers import BertJapaneseTokenizer
from transformers import BertForSequenceClassification
import  warnings
from transformers import get_linear_schedule_with_warmup

#何回大元のx解答を変更して検証するか
for oomoto_mawasu in range(10):
    #ここには位置番号が入ってる
    oomoto_kaburi = []
    gosaiten_kaburi = []
    
    #何回誤採点を作るか（そのx回で同じ答案は誤採点に選ばれない）
    #現状，ここのx回では毎回大元の答案が変わる
    for count_mawasu in range(1):
    
        subject_list = ["./rika","./shakai","./kokugo","./sas","./gpt_data/uscis","./gpt_data/seb","./gpt_data/sas", "./kanda", "./osaka"]
        question_list = ["/q1","/q2","/q3","/q4","/q5","/q6","/h29_q1","/s3", "/kq1", "/kq2", "/kq3", "/kq4", "/kq5", "/kq6", "/kq7", "/oq4", "/oq7", "/oq8","/q7","/q8", "/q9"]
    
        #print("科目は？　(1:理科  2:社会 3:kokugo 4:sas 5:gpt_uscis 6:gpt_seb 7:gpt_sas 8:kanda 9:osaka)")
        subject = subject_list[2]
        print()
        #print("採点する問いは？　(1:q1 2:q2 3:q3 4:q4 5:q5 6:q6 7:tms 8:s3 9:kq1 10:kq2 11:kq3 12:kq4 13:kq5 14:kq6, 15:kq7, 16:oq4 17:oq7 18:oq8)")
        question = question_list[20]
        print()
    
        base_path = subject + question
        write_path = base_path + "/result_total/"
        
        print("write_path: {}".format(write_path))
    
        #学習用解答データ
        read_path_text = base_path + question + 'kokugo_train_text.txt'
    
        #学習用に4倍した解答データ
        read_path_text_traingpt = base_path + question + 'kokugo_traingpt_text.txt'
    
        #誤採点検出用に10倍した解答データ
        read_path_text_testgpt = base_path + question + 'kokugo_testgpt_text.txt'
    
        #学習用得点データ
        read_path_score = base_path + question + 'kokugo_train_score.txt'
    
        #何分割にするか
        num = 4
        
        # ファイルから文章と得点を読み込む
        with open(read_path_text) as f:
            answer_text_before_all = [str(s.strip()) for s in f.readlines()]
            
        with open(read_path_text_traingpt) as f:
            answer_text_gpt_all = [str(s.strip()) for s in f.readlines()]
            
        with open(read_path_text_testgpt) as f:
            gosaiten_text_all = [str(s.strip()) for s in f.readlines()]
    
        with open(read_path_score) as f:
            graded_score_before_all = [str(s.strip()) for s in f.readlines()]
        

        
        gosaiten_iti = []
 
        def check_common_element(array1, array2):
            # フラグの初期値は0
            flag = 0
            
            # 二つの配列を比較し、一致する要素があればフラグを1にする
            for element in array1:
                if element in array2:
                    flag = 1
                    break  # 一致する要素が見つかったらループを終了
            
            # 結果を返す
            return flag
    
    
        #読み込んだ全体の解答文（array）からランダムにnum_of_gosaiten個（num_positions個）選ぶ
        #全体の解答の選んだ位置の得点を反転させる
        def select_random_positions(array, num_positions):
            if num_positions > len(array):
                print("Error: 要素数が不足しています。")
                return
    
            random_positions = random.sample(range(len(array)), num_positions)
            return random_positions
        
        
        num_of_oomoto = 100 #大元の数
        kaburiflag = 1
        while kaburiflag:
            selected_positions = select_random_positions(graded_score_before_all, num_of_oomoto)
            
            kaburiflag = check_common_element(oomoto_kaburi, selected_positions)
            
            if kaburiflag == 0:
                for k in selected_positions:
                    oomoto_kaburi.append(k)
        

        answer_text_before = []
        answer_text_gpt = []
        gosaiten_text = []
        graded_score_before = []
        
        #1解答あたり何解答をテスト用（誤採点検出用）として作成したか
        x_of_gpt_test = 10
        #拡張として何倍作成したか（学習用）
        x_of_gpt_train = 4
        
        for k in selected_positions:
            answer_text_before.append(answer_text_before_all[k])
            graded_score_before.append(graded_score_before_all[k])
            
            for l in range(x_of_gpt_train):
                answer_text_gpt.append(answer_text_gpt_all[k*x_of_gpt_train+l])
                
            for l in range(x_of_gpt_test):
                gosaiten_text.append(gosaiten_text_all[k*x_of_gpt_test+l])


        #kokokaeru
        num_of_gosaiten = 5 #誤採点の数
        kaburiflag = 1
        while kaburiflag == 1:
            selected_positions = select_random_positions(graded_score_before, num_of_gosaiten)
            print(selected_positions)
            kaburiflag = check_common_element(gosaiten_kaburi, selected_positions)
    
            if kaburiflag == 0:
                for k in selected_positions:
                    gosaiten_kaburi.append(k)
                    
        gosaiten_iti = selected_positions
    
        #全体の解答の大きさのフラグ配列つくり，選んだ位置のフラグを立てる
        flag = []
        for i in range(len(graded_score_before)):
            flag.append(0)
        for i in gosaiten_iti:
            flag[i] = 1
    
    
        print("誤採点に設定した解答とその番号，得点")
        for i in range(len(flag)):
            if flag[i] == 1:
                print(str(i), answer_text_before[i],"-",graded_score_before[i])
    
        for i in gosaiten_iti:
            #kokokaeru
            #今回は正解4点，不正解0点なので
            if graded_score_before[i] == "4":
                graded_score_before[i] = "0"
            elif graded_score_before[i] == "0":
                graded_score_before[i] = "4"
                
                
    
    
        #gptでx_of_gpt_train倍した分の得点を作る(元を反転後なのでこっちも反転したものがx倍される)
        #kokokaeru
        x_of_gpt_train = 4
        graded_score_gpt = []
        for i in graded_score_before:
            for k in range(x_of_gpt_train):
                graded_score_gpt.append(i)
    
        
        length_1 = len(answer_text_before)
    
        answer_text_before_n = []
        graded_score_before_n = []
        
        for i in range(num):
            if i == 0:
                answer_text_before_n.append(answer_text_before[:length_1//num])
                graded_score_before_n.append(graded_score_before[:length_1//num])
            elif i == num - 1:
                answer_text_before_n.append(answer_text_before[i * (length_1//num):])
                graded_score_before_n.append(graded_score_before[i * (length_1//num):])
            else:
                answer_text_before_n.append(answer_text_before[i * (length_1//num):(i+1) * (length_1//num)])
                graded_score_before_n.append(graded_score_before[i * (length_1//num):(i+1) * (length_1//num)])
        
        length_2 = (length_1 // num) * x_of_gpt_train
        
        answer_text_gpt_n = []
        graded_score_gpt_n = []
        for i in range(num):
            if i == 0:
                answer_text_gpt_n.append(answer_text_gpt[:length_2])
                graded_score_gpt_n.append(graded_score_gpt[:length_2])
            elif i == num - 1:
                answer_text_gpt_n.append(answer_text_gpt[i * length_2:])
                graded_score_gpt_n.append(graded_score_gpt[i * length_2:])
            else:
                answer_text_gpt_n.append(answer_text_gpt[i * length_2:(i+1) * length_2])
                graded_score_gpt_n.append(graded_score_gpt[i * length_2:(i+1) * length_2])
        

        length_3 = (length_1 // num) * x_of_gpt_test
    
        gosaiten_text_n = []
        for i in range(num):
            if i == 0:
                gosaiten_text_n.append(gosaiten_text[:length_3])
            elif i == num - 1:
                gosaiten_text_n.append(gosaiten_text[i * length_3:])
            else:
                gosaiten_text_n.append(gosaiten_text[i * length_3:(i+1) * length_3])
        
        
        
        
    
        #全体の解答とGPTでx_of_gpt_train倍した解答の文と得点の配列をそれぞれ結合させる
        graded_score = []
        answer_text = []

        for i in range(num):
            tmp_graded_score = []
            tmp_answer_text = []
            for k in range(num):
                if i == k:
                    continue
                else:
                    tmp_graded_score += graded_score_before_n[k]
                    tmp_graded_score += graded_score_gpt_n[k]
                    
                    tmp_answer_text += answer_text_before_n[k]
                    tmp_answer_text += answer_text_gpt_n[k]
            graded_score.append(tmp_graded_score)
            answer_text.append(tmp_answer_text)
    
        #GPTでx_of_gpt_test倍した分の得点を作る(元を反転後なのでこっちも反転したものがx_of_gpt_test倍される)
        #こっちはmodelの精度確認用（誤採点検出）
        gosaiten_score = []
        for i in graded_score_before:
            for k in range(x_of_gpt_test):
                gosaiten_score.append(i)
        
        gosaiten_score_n = []
        for i in range(num):
            if i == 0:
                gosaiten_score_n.append(gosaiten_score[:length_3])
            elif i == num - 1:
                gosaiten_score_n.append(gosaiten_score[i * length_3:])
            else:
                gosaiten_score_n.append(gosaiten_score[i * length_3:(i+1) * length_3])
        
        
  
        
        #エポック数の指定
        for num_of_epoch in range(1,6):
            loop_num = 1  # 実験の試行回数 モデルをいくつ作るか
            make_traindata_method = 2 # 訓練データの作り方は？　(1:クラスタの中心　2:ランダム)
        
            # クラスタ数（学習データ数）を決める
            # 1以上にするとk-means、１未満だと階層型クラスタリングを行う
            # WARNING: 現状、１未満にすると学習データが0個になってしまう
            cluster_num =  350
            BATCH_SIZE = 10
            if make_traindata_method == 1:
                if (cluster_num >= 1):
                    # バッチサイズごとに分けた場合、データが1個余るとエラーになる
                    if (cluster_num % BATCH_SIZE == 1 or (answer_num_gpt - cluster_num) % BATCH_SIZE == 1):
                        cluster_num += 1
        
        
            # モデルのパラメータを記述
            EMBEDDING_DIM = 768
            HIDDEN_DIM = 128
            TAG_SIZE = 2
        
        
            make_traindata_method = 2
            warnings.filterwarnings("ignore")
        
            start = time.time()
        
            #kokokaeru
            model_name = "cl-tohoku/bert-base-japanese-v2"
            #model_name = "bert-base-uncased"
            #model_name = "bert-base-cased"
            #"bert-base-japanese-v2"
            #"bert-base-japanese-whole-word-masking"
            #"bert-base-japanese-v2"
        
            
        
            for num_mawasi in range(num):
                loop_count = 0
                acc_list = []
                answer_num = len(graded_score[num_mawasi])
                print("----- {} -----".format(base_path))
        
                while(loop_count < loop_num):
                    loop_count += 1
                    tokenizer = AutoTokenizer.from_pretrained(model_name,is_fast=True)
                    
                    qwk_act = []
                    acc_act = []
                    #cluster_num0は学習データ数
                    cluster_num0 = answer_num
                    train_num = cluster_num0
        
                    
                    print("全解答数：　" + str(answer_num))
                    count_list = [0, 0]
                    for score in num_to_num(graded_score[num_mawasi]):
                        if score == 0:
                            count_list[0] += 1
                        elif score == 1:
                            count_list[1] += 1
                    
                    print("all data: [0, 1]={}".format(count_list))
        
                    # インデックスのリストで代入できるように変換する（処理を早くするため）
                    graded_score_gakusyu = np.array(graded_score[num_mawasi])
                    answer_text_gakusyu = np.array(answer_text[num_mawasi])
                    #print(graded_score_gakusyu)
                    #print(answer_text_gakusyu)
                    
                    train_vec = []
                    train_seq_vec = []
        
                    if make_traindata_method == 1:
                        print('クラスタの中心の解答を学習データにする')
                        print()
                        
                        # クラスタリングをする
                        index_list = clustering(cls_token_vec, answer_text_gakusyu, cluster_num)
                        
                    elif make_traindata_method == 2:
                        print('ランダムに解答を学習データにする')
                        print()
        
                        # インデックスをシャッフルする
                        np.random.seed(loop_count)
                        index_list = np.random.permutation(answer_num)
                    elif make_traindata_method == 3:
                        print("類似度で分類")
                        print()
                        index_list = clustering_cos(cls_token_vec,model_cls_vec,cluster_num)
                    elif make_traindata_method == 4:
                        print("ベクトル+類似度で分類")
                        print()
                        index_list = clustering_dis_cos(cls_token_vec,model_cls_vec,cluster_num)
                        
        
                    print("データを分ける")
                    encoded_data = tokenizer.batch_encode_plus(answer_text_gakusyu.tolist(), pad_to_max_length=True, add_special_tokens=True)
        
                    train_score = graded_score_gakusyu[index_list[:train_num]]
                    train_score = train_score.tolist()
                    train_text = answer_text_gakusyu[index_list[:train_num]]
                    train_text = train_text.tolist()
                    train_input_ids = torch.tensor(encoded_data["input_ids"])[index_list[:train_num]]
                    train_attention_mask = torch.tensor(encoded_data["attention_mask"])[index_list[:train_num]]
                    train_token_type_ids = torch.tensor(encoded_data["token_type_ids"])[index_list[:train_num]]
                    train_score_num = num_to_num(train_score)
                    print("学習データ")
                    
                    print('train_vec:{}, train_score:{}, train_text:{}, train_seq_vec:{}'.format(len(train_vec), len(train_score), len(train_text) ,len(train_seq_vec)))
                    print()
        
                    
                    count_list = [0, 0]
                    for score in train_score_num:
                        if score == 0:
                            count_list[0] += 1
                        elif score == 1:
                            count_list[1] += 1
        
                    
                    print("train data: [0, 1]={}".format(count_list))
                    lr_list = [5e-5,4e-5,3e-5,2e-5]
                    #lr_list = [5e-5]
                    learning_rates = 5e-5
                    max_qwk = 0
                    min_loss = 1000000000
                    for lr_count,LR in enumerate(lr_list):
        
                        model =  CommonLitModel(model_name,r = TAG_SIZE).to(device)  # to(device)でモデルがGPU対応する
        
                        no_decay = ['bias', 'LayerNorm.weight']
                        optimizer_grouped_parameters = [
                            {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                            {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
                        ]
                        optimizer = AdamW(optimizer_grouped_parameters, lr=LR)
        
                        model_path = base_path + "/model/model_num{}_{}.pth".format(num_mawasi+1, loop_count) # modelの保存先
        
                        loss_function = nn.CrossEntropyLoss()
                        
                        loss_epoch = []
                        acc = []
        
                        num_training_steps = ((train_num// BATCH_SIZE)+1)*5
                        scheduler = get_linear_schedule_with_warmup(optimizer, int(0.1*num_training_steps), num_training_steps)
                        # 学習
                        for epoch in range(num_of_epoch):
                            model.train() #学習モード
        
                            # バッチ化
                            text_batch = []
                            input_ids_batch = []
                            attention_mask_batch = []
                            token_type_ids_batch = []
                            category_batch = []
                            for i in range(0, train_num, BATCH_SIZE):
                                text_batch.append(train_text[i:i+BATCH_SIZE])
                                input_ids_batch.append(train_input_ids[i:i+BATCH_SIZE])
                                attention_mask_batch.append(train_attention_mask[i:i+BATCH_SIZE])
                                token_type_ids_batch.append(train_token_type_ids[i:i+BATCH_SIZE])
                                category_batch.append(train_score_num[i:i+BATCH_SIZE])
        
                            all_loss = 0
                            all_acc = 0
        
                            predict_list = []
                            for i in range(len(text_batch)):
        
                                batch_loss = 0
        
                                # 順伝搬させるtensorはGPUで処理させるためdevice=にGPUをセット
        
                                # category_tensor.size() = (batch_size × 1)なので、squeeze()
                                input_ids_tensor = torch.tensor(input_ids_batch[i], device=device)
                                attention_mask_tensor = torch.tensor(attention_mask_batch[i], device=device)
                                token_type_ids_tensor = torch.tensor(token_type_ids_batch[i], device=device)
                                category_tensor = torch.tensor(category_batch[i], device=device).squeeze()
        
                                logits,out,_ = model(input_ids_tensor,attention_mask=attention_mask_tensor,token_type_ids=token_type_ids_tensor)
                                model.zero_grad()
                                optimizer.zero_grad()
        
                                batch_loss = loss_function(logits, category_tensor)
                                batch_loss.backward()
        
                                optimizer.step()
                                scheduler.step()
        
                                all_loss += batch_loss.item()
                                predict_list.extend(num_to_num_after(torch.argmax(out,1)))
        
                            qwk = cohen_kappa_score(predict_list, train_score_num, weights='quadratic')
        
                            loss_epoch.append(all_loss)
                            acc.append(qwk)
        
                            print("epoch {: >3} | train_loss:{:.4f} , train_acc:{:.4f}".format(epoch+1, all_loss, qwk))
                            if(min_loss > all_loss):
                                print("Loss: save new weight at {} epoch".format(epoch+1))
                                torch.save(model.state_dict(), model_path) 
                                min_loss = all_loss
                                learning_rates = LR
                                losses_epoch = loss_epoch
                                accs = acc
                                
                                # dev_losses_epoch = dev_loss_epoch
                                # dev_accs = dev_acc
                            #torch.save(model.state_dict(), model_path)
                    
                    print("-------------------------------------------------------------------------")
                    print("|                                       "+ str(num_mawasi+1) + "の" + str(loop_count) + "回目終了！")
                    print("-------------------------------------------------------------------------\n")
                    # ----------------------------------------------------------loopの終わり---------------------------------------------------------
        
            elapsed_time = (time.time() - start) / 60
        
            print ("elapsed_time:{:.2f}".format(elapsed_time) + "[min]")
        
        
        
        
            #誤採点検出
        
            warnings.filterwarnings("ignore")
        
            loop_count = 0
            acc_list = []
            predict_list_mtm = []
        
            while(loop_count < loop_num):
                loop_count += 1
                predict_list_num_mawasi = []
                for num_mawasi in range(num):
                    
                    
                    tokenizer = AutoTokenizer.from_pretrained(model_name,is_fast=True)        
                    
                    answer_num = len(gosaiten_score_n[num_mawasi])
                    print("全解答数：　" + str(answer_num))
                    count_list = [0, 0]
                    for score in num_to_num(gosaiten_score_n[num_mawasi]):
                        if score == 0:
                            count_list[0] += 1
                        elif score == 1:
                            count_list[1] += 1
                    
                    print("all data: [0, 1]={}".format(count_list))
        
                    # インデックスのリストで代入できるように変換する（処理を早くするため）
                    gosaiten_score_test = np.array(gosaiten_score_n[num_mawasi])
                    gosaiten_text_test = np.array(gosaiten_text_n[num_mawasi])
                    print("データを分ける")
                    encoded_data = tokenizer.batch_encode_plus(gosaiten_text_test.tolist(), pad_to_max_length=True, add_special_tokens=True)
        
                    test_text = gosaiten_text_test
                    test_score = gosaiten_score_test
        
                    
                    test_input_ids = torch.tensor(encoded_data["input_ids"])
                    test_attention_mask = torch.tensor(encoded_data["attention_mask"])
                    test_token_type_ids = torch.tensor(encoded_data["token_type_ids"])
                    test_score_num = num_to_num(test_score)
                        
                    index_list = np.random.permutation(len(test_text))
                    
                    test_num = answer_num
                    
                        
        
                    # テストデータ
                    test_score = np.array(test_score).tolist()
                    test_text = np.array(test_text).tolist()
                
                    model =  CommonLitModel(model_name, r = TAG_SIZE).to(device)
        
                    #kokokaeru
                    model_path = base_path + "/model/model_num{}_{}.pth".format(num_mawasi+1,loop_count)
                    print(model_path)
                    # test
                    model.load_state_dict(torch.load(model_path))
        
        
                    # バッチ化
                    text_batch = []
                    test_cls = []
                    test_un = []
                    input_ids_batch = []
                    attention_mask_batch = []
                    token_type_ids_batch = []
                    category_batch = []
        
                    for i in range(0, test_num, BATCH_SIZE):
                        text_batch.append(test_text[i:i+BATCH_SIZE])
                        input_ids_batch.append(test_input_ids[i:i+BATCH_SIZE])
                        attention_mask_batch.append(test_attention_mask[i:i+BATCH_SIZE])
                        token_type_ids_batch.append(test_token_type_ids[i:i+BATCH_SIZE])
                        category_batch.append(test_score_num[i:i+BATCH_SIZE])
        
                    
                    predict_list = []
                    for i in range(len(text_batch)):
        
                        input_ids_tensor = torch.tensor(input_ids_batch[i], device=device)
                        attention_mask_tensor = torch.tensor(attention_mask_batch[i], device=device)
                        token_type_ids_tensor = torch.tensor(token_type_ids_batch[i], device=device)
                        category_tensor = torch.tensor(category_batch[i], device=device).squeeze()
                        logits,out,out1 = model(input_ids_tensor,attention_mask_tensor,token_type_ids_tensor)
        
                        test_un.extend(out.tolist())
                        test_cls.extend(out1.tolist())
        
                        predict_list.extend(num_to_num_after(torch.argmax(out,1)))
                    predict_list_num_mawasi.append(predict_list)
        
                predict_list_mtm.append(sum(predict_list_num_mawasi,[])) 
        
                acc_list.append(accuracy_score(predict_list_mtm[loop_count-1], num_to_num(sum(gosaiten_score_n,[]) )))
                
                print("acc_list",acc_list)
        
                print("-------------------------------------------------------------------------")
                print("|                                       "+ str(loop_count) + "の" + str(num_mawasi+1) + "回目終了！")
                print("-------------------------------------------------------------------------\n")
                # ----------------------------------------------------------loopの終わり--------------------------------------------------------
                
                
                
                
                
                
                
            list_of_miss_count_list_ave = []
            list_of_true_miss_count_list_ave = []
        
            fuitti_count_max = 1
        
            graded_score_before_copy = []
        
            for i in graded_score_before:
                graded_score_before_copy.append(i)
        
        
            for n in range(len(graded_score_before_copy)):
                if graded_score_before_copy[n] != '0':
                    graded_score_before_copy[n] = 1
                else:
                    graded_score_before_copy[n] = 0
        
        
            print(len(predict_list_mtm[0]))

            kensyutu_dekita_touan_list = []
            #GPTでx_of_gpt_test倍に拡張したから最大でもfuitti_count_maxはx_of_gpt_testだから
            for fuitti_turn in range(x_of_gpt_test):
                print("fuitti_count_max=",fuitti_count_max)
                
                miss_count_list_ave = 0
                true_miss_count_list_ave = 0
                #今回はmodelがloop_num個だから
                for model_turn in range(loop_num):
                    print("model_name=",model_turn+1)
                    # モデル絞る時は下のコメントアウト解除，割る数も変える（下のところ）
                    # if (model_turn+1) != 1:
                    #     continue
                    
                    #GPTの予測結果
                    yosoku_kekka = predict_list_mtm[model_turn]
                    miss_count_list = []
                    true_miss_count_list = []
                    
                    #今回は誤採点リストを1作ったから
                    for i in range(1):
                        
                        #誤採点と認識したものカウント
                        miss_count = 0
                        #そのうちの真誤採点カウント
                        true_miss_count = 0
                        m = 0
                        for k in range(len(graded_score_before_copy)):
                            #不一致カウント
                            fuitti_count = 0
                            #今回はGPTでx_of_gpt_test倍に拡張したから
                            for l in range(x_of_gpt_test):
                                if graded_score_before_copy[k] != yosoku_kekka[m]:
                                    fuitti_count+=1
                                m+=1
                            if fuitti_count >= fuitti_count_max:
                                miss_count += 1
                                if int(flag[k]) == 1:
                                    print(answer_text_before[k])
                                    true_miss_count += 1
                                    ####
                                    if fuitti_count_max == 1:
                                        kensyutu_dekita_touan_list.append(answer_text_before[k])
                        miss_count_list.append(miss_count)
                        true_miss_count_list.append(true_miss_count)
                        miss_count_list_ave += miss_count
                        true_miss_count_list_ave += true_miss_count
        
                    print(miss_count_list)
                    print(true_miss_count_list) 
                    
                #今回は誤採点リストを1作り，modelはloop_num個だから
                miss_count_list_ave /= loop_num
                true_miss_count_list_ave /= loop_num
                list_of_miss_count_list_ave.append(miss_count_list_ave)
                list_of_true_miss_count_list_ave.append(true_miss_count_list_ave)
        
                print()
                print(miss_count_list_ave)
                print(true_miss_count_list_ave) 
                print()   
                fuitti_count_max += 1   
                
            write_result_path = base_path + "kekka_"+str(count_mawasu+1)+"oomoto_"+str(oomoto_mawasu+1)+"-bestepoch"+str(num_of_epoch)+".txt"
            with open(write_result_path, mode='w') as f:
                f.write("誤採点の一覧\n")
                for i in range(len(flag)):
                    if flag[i] == 1:
                        f.write(str(i)+":"+answer_text_before[i]+"\n")
                        
                f.write("検出できた誤採点\n")
                for i in range(len(kensyutu_dekita_touan_list)):
                    f.write(kensyutu_dekita_touan_list[i]+"\n")
                    
                f.write("検出数\n")
                for i in range(len(list_of_miss_count_list_ave)):
                    f.write(str(list_of_miss_count_list_ave[i])+"\n")
                    
                f.write("検出したうち本当の誤採点数\n")
                for i in range(len(list_of_true_miss_count_list_ave)):
                    f.write(str(list_of_true_miss_count_list_ave[i])+"\n")